In [39]:
import requests
import pandas as pd

In [4]:


params = {
    'appname': 'amcross'  
    ,'profile': 'full'
    ,'preset': 'latest'
    ,'limit': 3
    ,'query[fields][]':'format.name'
    ,'query[value]':'Situation Report'
    #if I remove this, seems like it returns everything
    #,'fields[include][]':['source',"format",'title',"body",'disaster.glide','primary_country']
}


api_endpoint = 'https://api.reliefweb.int/v1/reports?appname=amcross'

# Make the API request
response = requests.get(api_endpoint, params=params)

# Check the status of the response
if response.status_code == 200:
    # Parse and use the response data (in JSON format)
    data = response.json()
    situation_reports = data['data']

else:
    print(f"Error: {response.status_code} - {response.text}")

In [10]:
json_data = response.json()['data']
len(json_data)

3

In [44]:
df_reliefweb_situation_report = pd.DataFrame(columns = ['record_type','source_url','glide_id','original_text','reference_url','authoring_ord','reported_date'])
for j in json_data:

    #parse 
    reference_url = j['href']
        
    j = j['fields']
    
    glide_id = None
    disaster = j.get('disaster')
    if disaster:
        glide_id = disaster[0]['glide']

    
    rec_id = j['id']
    title = j['title']
    original_text = j['body']
    link_to_doc = j['url_alias']
    file_url = j['file'][0]['url']
    primary_country_iso3 = j['primary_country']['iso3']
    primary_country = j['primary_country']['shortname']
    author_org = j['source'][0]['shortname']
    report_date = j['date']['original']
   
    original_text_list = original_text.split("\n\n")
    for o in original_text_list:
        row = ['situation report',api_reference,glide_id,o,reference_url,author_org,report_date]
        df_reliefweb_situation_report.loc[len(df_reliefweb_situation_report)] = row

In [45]:
df_reliefweb_situation_report

,record_type,source_url,glide_id,original_text,reference_url,authoring_ord,reported_date
0,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,**Highlights**,https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
1,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,"- As of 31 October 2023, UNICEF, together with...",https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
2,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,"- Through UNICEF support, 3,890,459 people hav...",https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
3,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,"- UNICEF supported 901,951 children to access ...",https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
4,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,"- Over 98,000 individuals were reached with he...",https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
5,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,- More than 1.3 million earthquake affected ch...,https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
6,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,- UNICEF has a funding gap of over US$ 56 mill...,https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
7,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,\n**Situation Overview & Humanitarian Needs**,https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
8,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,While the data on the total number of people ...,https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
9,situation report,https://api.reliefweb.int/v1/reports/4017643,EQ-2023-000015-TUR,"As winter approaches, with deteriorating, cold...",https://api.reliefweb.int/v1/reports/4017650,UNICEF,2023-11-24T00:00:00+00:00
